In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time

In [ ]:
df = pd.read_csv('Todos.csv', sep=';')
df = df.drop(['Operacoes_medicoes', 'Temperatura_ensaio','Operacoes_anual', 'Nivel de atrito'], axis=1)
X = df.loc[:, ['Distancia_medicao', 'Lado', 'Remocao_anterior', 'Idade_pavimento_meses', 'Umidade_AISWEB', 'Operacoes_remocoes']]
Y = df['Atrito']

## Dividindo os conjuntos de dados entre 90% para Treinamento e 10% para Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

## Pré-processando os dados de entrada utilizando o pré-processamento Standard Scaler

\begin{equation}
	z = \frac{(x - \overline{x})}{s}
\end{equation}
$z$: valor normalizado;<br>
$x$: valor a ser normalizado;<br>
$\overline{x}$: média;<br>
$s$: desvio padrão.

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
scaler_x = StandardScaler()
scaler_x.fit(X_train) # Ajustando os dados para a média e desvio padrão do conjunto de teste
X_train = scaler_x.transform(X_train) # Aplicando a transformação no conjunto de treinamento
X_test = scaler_x.transform(X_test) # O conjunto de teste é pré-processado de acordo com o conjunto de treinamento

# Testando 1 camada

In [ ]:
inicio = time.time()

quantidade_arq_testadas = 0

mae_teste_min = 1

for i in np.arange(1, 101):
    for k in ['logistic', 'tanh', 'relu']:
        for m in np.arange(0, 11):
            reg = MLPRegressor(
                hidden_layer_sizes=(i,),
                activation=k,
                solver='lbfgs',
                alpha=0.1,
                random_state=m,
                max_iter=600)
            
            reg.fit(X_train, y_train)
            
            armazenando_score_teste = reg.score(X_test, y_test)
                
            armazenando_score_treinamento = reg.score(X_train, y_train)
                
            quantidade_arq_testadas += 1
                        
            ultimo_neuronio_camada_1 = i
                
            ultima_funcao_ativacao = k
                
            ultimo_random_state = m
                
            mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                        
            print('Cam 1: %.i' %i, 'F. Ativ.: %s' %k, 'Rnd St: %.i' %m,'Arq_testadas: %.0i' %quantidade_arq_testadas, 'R² treino: %.2f' %(armazenando_score_treinamento*100), 'R² teste: %.2f' %(armazenando_score_teste*100), 'mae_teste: %.3f' %mae_teste)
                        
            if mae_teste < mae_teste_min: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                mae_teste_min = mae_teste
                mse_teste = mean_squared_error(y_test, reg.predict(X_test))
                armazenando_score_teste_max = armazenando_score_teste
                neuronios_camada_1 = i
                funcao_ativacao = k
                random_state = m
            
            
            fim = time.time()
            tempo_total = fim - inicio
print('mae_teste_min: %.3f\n' %mae_teste_min, 'armazenando_score_teste_max: %.2f' %(armazenando_score_teste_max*100))

## Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_teste)
print('Erro Médio Absoluto do teste: %.3f' %mae_teste_min)

## Salvando os resultados

In [ ]:
np.savetxt('M2_1_camada.txt',
    np.c_[armazenando_score_teste_max,
          mae_teste_min,
          mse_teste,
          neuronios_camada_1,
          random_state,
          tempo_total],
    fmt='%.4f',
    newline='\n',
    header='#armazenando_score_teste_max #mae_teste_min #mse_teste #neuronios_camada_1 #random_state #tempo_total',
    comments='funcao_ativacao %s\n' %funcao_ativacao)

# Testando 2 camadas

In [ ]:
inicio = time.time()

quantidade_arq_testadas = 0

mae_teste_min = 1

for j in np.arange(1, 101):
    for i in np.arange(1, 101):
        for k in ['logistic', 'tanh', 'relu']:
            for m in np.arange(0, 11):
                reg = MLPRegressor(
                    hidden_layer_sizes=(i,j),
                    activation=k,
                    solver='lbfgs',
                    alpha=0.1,
                    random_state=m,
                    max_iter=600)
            
                reg.fit(X_train, y_train)
            
                armazenando_score_teste = reg.score(X_test, y_test)
                
                armazenando_score_treinamento = reg.score(X_train, y_train)
                
                quantidade_arq_testadas += 1
                        
                ultimo_neuronio_camada_1 = i
                        
                ultimo_neuronio_camada_2 = j
                
                ultima_funcao_ativacao = k
                
                ultimo_random_state = m
                
                mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                        
                print('Cam 1: %.i' %i, 'Cam 2: %.i' %j, 'F. Ativ.: %s' %k, 'Rnd St: %.i' %m,'Arq_testadas: %.0i' %quantidade_arq_testadas, 'R² treino: %.2f' %(armazenando_score_treinamento*100), 'R² teste: %.2f' %(armazenando_score_teste*100), 'mae_teste: %.3f' %mae_teste)
                        
                if mae_teste < mae_teste_min: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                    mae_teste_min = mae_teste
                    mse_teste = mean_squared_error(y_test, reg.predict(X_test))
                    armazenando_score_teste_max = armazenando_score_teste
                    neuronios_camada_1 = i
                    neuronios_camada_2 = j
                    funcao_ativacao = k
                    random_state = m
            
            
                fim = time.time()
                tempo_total = fim - inicio
print('mae_teste_min: %.3f\n' %mae_teste_min, 'armazenando_score_teste_max: %.2f' %(armazenando_score_teste_max*100))

## Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_teste)
print('Erro Médio Absoluto do teste: %.3f' %mae_teste_min)

## Salvando os resultados

In [ ]:
np.savetxt('M2_2_camadas.txt',
    np.c_[armazenando_score_teste_max,
          mae_teste_min,
          mse_teste,
          neuronios_camada_1,
          neuronios_camada_2,
          random_state,
          tempo_total],
    fmt='%.4f',
    newline='\n',
    header='#armazenando_score_teste_max #mae_teste_min #mse_teste #neuronios_camada_1 #neuronios_camada_2 #random_state #tempo_total',
    comments='funcao_ativacao %s\n' %funcao_ativacao)

## Pré-processando os dados de entrada utilizando o pré-processamento MinMaxScaler

\begin{equation}
 z = \frac{(x - x_\textit{min}) \times (d_2 - d_1)}{(x_\textit{max} - x_\textit{min})} + d_1
\end{equation}
$z$: valor normalizado;

$x$: valor a ser normalizado;

$x_{max}$ e $x_{min}$: variação do valor de $x$;

$d_1$ e $d_2$: limite ao qual o valor $x$ será reduzido.

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

### Dividindo o cojunto de dados em 90% Treinamento e 10% Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

### Pré-processamento dos dados
**NOTA: Os dados de saída (coeficiente de atrito) foram normalizados**

In [ ]:
train = pd.concat([pd.DataFrame(X_train, columns=X.columns), pd.Series(y_train)], axis=1)
mas_scaler = MaxAbsScaler()
mas_scaler.fit(train)
train = mas_scaler.transform(train)
X_train, y_train = pd.DataFrame(train[:, :6], columns=X.columns), pd.Series(train[:, 6])

test = pd.concat([pd.DataFrame(X_test, columns=X.columns), pd.Series(y_test)], axis=1)
test = mas_scaler.transform(test)
X_test, y_test = pd.DataFrame(test[:, :6], columns=X.columns), pd.Series(test[:, 6])

### Estatítisca descritiva do conjunto de entrada do treinamento

In [ ]:
X_train.describe().round(2)

### Estatítisca descritiva do conjunto de entrada do teste

In [ ]:
X_test.describe().round(2)

# Testando 1 camada

In [ ]:
inicio = time.time()

quantidade_arq_testadas = 0

mae_teste_min = 1

armazenando_score_teste_max = 0

for i in np.arange(1, 101):
    for k in ['logistic', 'tanh', 'relu']:
        for m in np.arange(0, 11):
            reg = MLPRegressor(
                hidden_layer_sizes=(i,),
                activation=k,
                solver='lbfgs',
                alpha=0.1,
                random_state=m,
                max_iter=600)
            
            reg.fit(X_train, y_train)
            
            armazenando_score_teste = reg.score(X_test, y_test)
                
            armazenando_score_treinamento = reg.score(X_train, y_train)
                
            quantidade_arq_testadas += 1
                        
            ultimo_neuronio_camada_1 = i
                
            ultima_funcao_ativacao = k
                
            ultimo_random_state = m
                
            mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                        
            print('Cam 1: %.i' %i, 'F. Ativ.: %s' %k, 'Rnd St: %.i' %m,'Arq_testadas: %.0i' %quantidade_arq_testadas, 'R² treino: %.2f' %(armazenando_score_treinamento*100), 'R² teste: %.2f' %(armazenando_score_teste*100), 'mae_teste: %.3f' %mae_teste)
                        
            if mae_teste < mae_teste_min: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                mae_teste_min = mae_teste
                mse_teste = mean_squared_error(y_test, reg.predict(X_test))
                armazenando_score_teste_max = armazenando_score_teste
                neuronios_camada_1 = i
                funcao_ativacao = k
                random_state = m
            
            
            fim = time.time()
            tempo_total = fim - inicio
print('mae_teste_min: %.3f\n' %mae_teste_min, 'armazenando_score_teste_max: %.2f' %(armazenando_score_teste_max*100))

## Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_teste)
print('Erro Médio Absoluto do teste: %.3f' %mae_teste_min)

## Salvando os resultados

In [ ]:
np.savetxt('M2_1_camada_mms.txt',
    np.c_[armazenando_score_teste_max,
          mae_teste_min,
          mse_teste,
          neuronios_camada_1,
          random_state,
          tempo_total],
    fmt='%.4f',
    newline='\n',
    header='#armazenando_score_teste_max #mae_teste_min #mse_teste #neuronios_camada_1 #random_state #tempo_total',
    comments='funcao_ativacao %s\n' %funcao_ativacao)

# Testando 2 camadas

In [ ]:
inicio = time.time()

quantidade_arq_testadas = 0

mae_teste_min = 1

for j in np.arange(1, 101):
    for i in np.arange(1, 101):
        for k in ['logistic', 'tanh', 'relu']:
            for m in np.arange(0, 11):
                reg = MLPRegressor(
                    hidden_layer_sizes=(i,j),
                    activation=k,
                    solver='lbfgs',
                    alpha=0.1,
                    random_state=m,
                    max_iter=600)
            
                reg.fit(X_train, y_train)
            
                armazenando_score_teste = reg.score(X_test, y_test)
                
                armazenando_score_treinamento = reg.score(X_train, y_train)
                
                quantidade_arq_testadas += 1
                        
                ultimo_neuronio_camada_1 = i
                        
                ultimo_neuronio_camada_2 = j
                
                ultima_funcao_ativacao = k
                
                ultimo_random_state = m
                
                mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                        
                print('Cam 1: %.i' %i, 'Cam 2: %.i' %j, 'F. Ativ.: %s' %k, 'Rnd St: %.i' %m,'Arq_testadas: %.0i' %quantidade_arq_testadas, 'R² treino: %.2f' %(armazenando_score_treinamento*100), 'R² teste: %.2f' %(armazenando_score_teste*100), 'mae_teste: %.3f' %mae_teste)
                        
                if mae_teste < mae_teste_min: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                    mae_teste_min = mae_teste
                    mse_teste = mean_squared_error(y_test, reg.predict(X_test))
                    armazenando_score_teste_max = armazenando_score_teste
                    neuronios_camada_1 = i
                    neuronios_camada_2 = j
                    funcao_ativacao = k
                    random_state = m
            
            
                fim = time.time()
                tempo_total = fim - inicio
print('mae_teste_min: %.3f\n' %mae_teste_min, 'armazenando_score_teste_max: %.2f' %(armazenando_score_teste_max*100))

## Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_teste)
print('Erro Médio Absoluto do teste: %.3f' %mae_teste_min)

## Salvando os resultados

In [ ]:
np.savetxt('M2_2_camadas_mms.txt',
    np.c_[armazenando_score_teste_max,
          mae_teste_min,
          mse_teste,
          neuronios_camada_1,
          neuronios_camada_2,
          random_state,
          tempo_total],
    fmt='%.4f',
    newline='\n',
    header='#armazenando_score_teste_max #mae_teste_min #mse_teste #neuronios_camada_1 #neuronios_camada_2 #random_state #tempo_total',
    comments='funcao_ativacao %s\n' %funcao_ativacao)

## Pré-processando os dados de entrada utilizando o pré-processamento MaxAbsScaler

Fonte: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#sklearn.preprocessing.MaxAbsScaler

### Dividindo o cojunto de dados em 90% Treinamento e 10% Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

### Pré-processamento dos dados
**NOTA: Os dados de saída (coeficiente de atrito) foram normalizados**

In [ ]:
train = pd.concat([pd.DataFrame(X_train, columns=X.columns), pd.Series(y_train)], axis=1)
mas_scaler = MinMaxScaler()
mas_scaler.fit(train)
train = mas_scaler.transform(train)
X_train, y_train = pd.DataFrame(train[:, :6], columns=X.columns), pd.Series(train[:, 6])

test = pd.concat([pd.DataFrame(X_test, columns=X.columns), pd.Series(y_test)], axis=1)
test = mas_scaler.transform(test)
X_test, y_test = pd.DataFrame(test[:, :6], columns=X.columns), pd.Series(test[:, 6])

### Estatítisca descritiva do conjunto de entrada do treinamento

In [ ]:
X_train.describe().round(2)

### Estatítisca descritiva do conjunto de entrada do teste

In [ ]:
X_test.describe().round(2)

# Testando 1 camada

In [ ]:
inicio = time.time()

quantidade_arq_testadas = 0

mae_teste_min = 1

armazenando_score_teste_max = 0

for i in np.arange(1, 101):
    for k in ['logistic', 'tanh', 'relu']:
        for m in np.arange(0, 11):
            reg = MLPRegressor(
                hidden_layer_sizes=(i,),
                activation=k,
                solver='lbfgs',
                alpha=0.1,
                random_state=m,
                max_iter=600)
            
            reg.fit(X_train, y_train)
            
            armazenando_score_teste = reg.score(X_test, y_test)
                
            armazenando_score_treinamento = reg.score(X_train, y_train)
                
            quantidade_arq_testadas += 1
                        
            ultimo_neuronio_camada_1 = i
                
            ultima_funcao_ativacao = k
                
            ultimo_random_state = m
                
            mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                        
            print('Cam 1: %.i' %i, 'F. Ativ.: %s' %k, 'Rnd St: %.i' %m,'Arq_testadas: %.0i' %quantidade_arq_testadas, 'R² treino: %.2f' %(armazenando_score_treinamento*100), 'R² teste: %.2f' %(armazenando_score_teste*100), 'mae_teste: %.3f' %mae_teste)
                        
            if mae_teste < mae_teste_min: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                mae_teste_min = mae_teste
                mse_teste = mean_squared_error(y_test, reg.predict(X_test))
                armazenando_score_teste_max = armazenando_score_teste
                neuronios_camada_1 = i
                funcao_ativacao = k
                random_state = m
            
            
            fim = time.time()
            tempo_total = fim - inicio
print('mae_teste_min: %.3f\n' %mae_teste_min, 'armazenando_score_teste_max: %.2f' %(armazenando_score_teste_max*100))

## Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_teste)
print('Erro Médio Absoluto do teste: %.3f' %mae_teste_min)

## Salvando os resultados

In [ ]:
np.savetxt('M2_1_camada_mas.txt',
    np.c_[armazenando_score_teste_max,
          mae_teste_min,
          mse_teste,
          neuronios_camada_1,
          random_state,
          tempo_total],
    fmt='%.4f',
    newline='\n',
    header='#armazenando_score_teste_max #mae_teste_min #mse_teste #neuronios_camada_1 #random_state #tempo_total',
    comments='funcao_ativacao %s\n' %funcao_ativacao)

# Testando 2 camadas

In [ ]:
inicio = time.time()

quantidade_arq_testadas = 0

mae_teste_min = 1

for j in np.arange(1, 101):
    for i in np.arange(1, 101):
        for k in ['logistic', 'tanh', 'relu']:
            for m in np.arange(0, 11):
                reg = MLPRegressor(
                    hidden_layer_sizes=(i,j),
                    activation=k,
                    solver='lbfgs',
                    alpha=0.1,
                    random_state=m,
                    max_iter=600)
            
                reg.fit(X_train, y_train)
            
                armazenando_score_teste = reg.score(X_test, y_test)
                
                armazenando_score_treinamento = reg.score(X_train, y_train)
                
                quantidade_arq_testadas += 1
                        
                ultimo_neuronio_camada_1 = i
                        
                ultimo_neuronio_camada_2 = j
                
                ultima_funcao_ativacao = k
                
                ultimo_random_state = m
                
                mae_teste = mean_absolute_error(y_test, reg.predict(X_test))
                        
                print('Cam 1: %.i' %i, 'Cam 2: %.i' %j, 'F. Ativ.: %s' %k, 'Rnd St: %.i' %m,'Arq_testadas: %.0i' %quantidade_arq_testadas, 'R² treino: %.2f' %(armazenando_score_treinamento*100), 'R² teste: %.2f' %(armazenando_score_teste*100), 'mae_teste: %.3f' %mae_teste)
                        
                if mae_teste < mae_teste_min: # Esse if é utilizado para salvar apenas o melhor valor de Escore no teste e assim otimizar o algoritmo
                    mae_teste_min = mae_teste
                    mse_teste = mean_squared_error(y_test, reg.predict(X_test))
                    armazenando_score_teste_max = armazenando_score_teste
                    neuronios_camada_1 = i
                    neuronios_camada_2 = j
                    funcao_ativacao = k
                    random_state = m
            
            
                fim = time.time()
                tempo_total = fim - inicio
print('mae_teste_min: %.3f\n' %mae_teste_min, 'armazenando_score_teste_max: %.2f' %(armazenando_score_teste_max*100))

## Apresentando os melhores resultados

In [ ]:
print('Taxa de acerto do teste: %.2f' %(armazenando_score_teste_max*100))
print('Erro Quadrático Médio do teste: %.3f' %mse_teste)
print('Erro Médio Absoluto do teste: %.3f' %mae_teste_min)

## Salvando os resultados

In [ ]:
np.savetxt('M2_2_camadas_mas.txt',
    np.c_[armazenando_score_teste_max,
          mae_teste_min,
          mse_teste,
          neuronios_camada_1,
          neuronios_camada_2,
          random_state,
          tempo_total],
    fmt='%.4f',
    newline='\n',
    header='#armazenando_score_teste_max #mae_teste_min #mse_teste #neuronios_camada_1 #neuronios_camada_2 #random_state #tempo_total',
    comments='funcao_ativacao %s\n' %funcao_ativacao)